<a href="https://colab.research.google.com/github/Khronophobia/stable-diffusion-webui-Colab/blob/main/a1111-webui-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stable Diffusion Colab

This notebook runs [A1111 Stable Diffusion WebUI.](https://github.com/AUTOMATIC1111/stable-diffusion-webui)

Follow the steps below to run Stable Diffusion.

### Changelog: (YYYY/MM/DD)
- 2023/04/15 Revised instructions so that it's (hopefully) clearer. Added the option to download models in grops
- 2023/04/14 Merged `Install WebUI` and `Install Dependencies` back to a single cell
- 2023/04/12 Add v2 models. Revised instructions. Add CanvasZoom
- 2023/04/11 Change repository name. Add support for PyTorch 2.0.0. Split 'Install WebUI' to 3 cells
- 2023/04/10 Add Midnight Mixes
- 2023/04/09 Add Civitai Helper
- 2023/04/09 Add Anything V5

# 1. Install WebUI

In [ ]:
#@title ## Install Dependencies
#@markdown ##### Clone A1111 WebUI from GitHub and install required dependencies.
webui_version = "Latest" #@param ["Latest", "Stable", "Custom"]
#@markdown ##### Select which version to install.
commit = "" #@param {type:"string"}
#@markdown ##### Specify the commit if you selected Custom.
torch_version = "PyTorch 2.0.0" #@param ["PyTorch 2.0.0", "PyTorch 1.13.1"]
#@markdown ##### Select PyTorch version. Colab officially supports 2.0.0, while the WebUI officially supports 1.13.1. Do what you will.

import os, glob

if webui_version == "Latest":
  commit = 'master'
elif webui_version == "Stable":
  commit = 'a9fed7c364061ae6efb37f797b6b522cb3cf7aa2'

!mkdir -p '/content/'
os.chdir('/content/')
%cd /content/

if "LD_PRELOAD" not in os.environ:
  # This fixes memory leaks apparently. No idea what it does. Shoutout to falsonerd from GitHub!
  !wget -qq --show-progress http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
  !wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
  !wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
  !wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
  !apt install -qq libunwind8-dev
  !dpkg -i *.deb
  !rm *.deb
  os.environ["LD_PRELOAD"] = "libtcmalloc.so"

!apt-get -y install -qq aria2
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd /content/stable-diffusion-webui
!git checkout -f {commit}

# There's probably a better way to do this, but if it works it works (I hope at least it works anyway.)
if torch_version == "PyTorch 2.0.0":
  !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
  !pip install xformers==0.0.17
  !sed -i 's/pip=22\.2\.2/pip=23.0/; s/cudatoolkit=11\.3/cudatoolkit=11.8/; s/pytorch=1\.12\.1/pytorch=2.0/; s/torchvision=0\.13\.1/torchvision=0\.15/; s/numpy=1\.23\.1/numpy=1.23/' /content/stable-diffusion-webui/environment-wsl2.yaml
  !sed -i 's|pip install torch==1.13.1+cu117 torchvision==0\.14\.1+cu117 --extra-index-url https://download\.pytorch\.org/whl/cu117|pip install torch torchvision --extra-index-url https://download\.pytorch\.org/whl/cu118|; s|xformers==0\.0\.16rc425|xformers==0\.0\.17|' /content/stable-diffusion-webui/launch.py
  !sed -i '1iastunparse' /content/stable-diffusion-webui/requirements.txt
  !sed -i 's/accelerate==0\.12\.0/accelerate==0\.18\.0/; s/numpy==1\.23\.3/numpy==1\.23\.5/' /content/stable-diffusion-webui/requirements_versions.txt
else:
  !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117
  !pip install xformers==0.0.16rc425
  !sed -i 's/pip=23.0/pip=22\.2\.2/; s/cudatoolkit=11.8/cudatoolkit=11\.3/; s/pytorch=2.0/pytorch=1\.12\.1/; s/torchvision=0\.15/torchvision=0\.13\.1/; s/numpy=1.23/numpy=1\.23\.1/' /content/stable-diffusion-webui/environment-wsl2.yaml
  !sed -i 's|pip install torch torchvision --extra-index-url https://download\.pytorch\.org/whl/cu118|pip install torch==1.13.1+cu117 torchvision==0\.14\.1+cu117 --extra-index-url https://download\.pytorch\.org/whl/cu117|; s|xformers==0\.0\.17|xformers==0\.0\.16rc425|' /content/stable-diffusion-webui/launch.py
  !sed -i '/astunparse/d' /content/stable-diffusion-webui/requirements.txt
  !sed -i 's/accelerate==0\.18\.0/accelerate==0\.12\.0/; s/numpy==1\.23\.5/numpy==1\.23\.3/' /content/stable-diffusion-webui/requirements_versions.txt

!python launch.py --exit

In [ ]:
#@title ## (Optional) Mount Google Drive
#@markdown ##### Link this Colab to Google Drive and save your outputs there
output_path = "SD_WEBUI" #@param {type:"string"}
#@markdown ##### Where outputs will be saved.

from google.colab import drive
drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/' + output_path
# make necessary directories if not exists
!mkdir -p {output_path}/outputs
!mkdir -p {output_path}/models
!mkdir -p {output_path}/upscalers/ESRGAN
!mkdir -p {output_path}/upscalers/SwinIR
!mkdir -p {output_path}/hypernetworks
!mkdir -p {output_path}/embeddings
!mkdir -p {output_path}/Lora

if not os.path.exists(output_path + '/ui-config.json'):
  print("Create new ui-config.json file.")    
  !wget https://github.com/Khronophobia/stable-diffusion-webui-Colab/raw/main/ui-config.json -O {output_path + '/ui-config.json'}
if not os.path.exists(output_path + '/config.json'):
  print("Create new config.json file.")    
  !wget https://github.com/Khronophobia/stable-diffusion-webui-Colab/raw/main/config.json -O {output_path + '/config.json'}

!ln -s {output_path + '/ui-config.json'} /content/stable-diffusion-webui/
!ln -s {output_path + '/config.json'} /content/stable-diffusion-webui/
!ln -s {output_path}/outputs

# save parameter file in google drive
if not os.path.exists(output_path + '/params.txt'):
  !touch {output_path + '/params.txt'}
!ln -s {output_path}/params.txt

# Embeddings in drive
%cd /content/stable-diffusion-webui
!rm -rf embeddings
!ln -s {output_path}/embeddings

# Lora in drive
%cd /content/stable-diffusion-webui/models
!rm -rf Lora
!ln -s {output_path}/Lora

# Hypernetworks in drive
%cd /content/stable-diffusion-webui/models
!rm -rf hypernetworks
!ln -s {output_path}/hypernetworks

# link all models in the models folder
%cd /content/stable-diffusion-webui/models/Stable-diffusion
models_in_google_drive = glob.glob(output_path + '/models/*')
print('Models in Google Drive: %s'%models_in_google_drive)
for f in models_in_google_drive:
  !ln -s {f}

# link all upscalers in the model folder
%cd /content/stable-diffusion-webui/models
upscalers_in_google_drive = glob.glob(output_path + '/upscalers/**/*.*', recursive=True)
print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)
for f in upscalers_in_google_drive:
  upscaler_relpath = os.path.relpath(f, output_path + "/upscalers")
  upscaler_folder = os.path.dirname(upscaler_relpath)
  !mkdir -p {upscaler_folder}
  !ln -s {f} {upscaler_folder}

# 2. Download Models and Extensions

In [ ]:
#@title ## Download the models you are going to use from the methods below
#@markdown ### Method 1: Choose from dropdown
Download_Model = "Stable Diffusion v1.5" #@param ["None", "Stable Diffusion v1.5", "Stable Diffusion v1.5 (inpainting)", "Realistic Vision v2.0", "ChilloutMix", "Deliberate", "Dreamshaper V3", "Dreamshaper V4", "Waifu Diffusion 1.3", "Anything V3", "Anything V4.5", "Anything V5", "Counterfeit V2.5", "AbyssOrangeMix2 SFW (OrangeMixs)", "AbyssOrangeMix3 (OrangeMixs)", "AbyssOrangeMix3A1 (OrangeMixs)", "AbyssOrangeMix3A1B (OrangeMixs)", "AbyssOrangeMix3A2 (OrangeMixs)", "AbyssOrangeMix3A3 (OrangeMixs)", "BloodOrangeMix (OrangeMixs)", "Midnight Maple (Midnight Mixes)", "Midnight Melt (Midnight Mixes)", "Midnight Mixer Melt (Midnight Mixes)", "Stable Diffusion v2.1", "Stable Diffusion v2.1 768", "Stable Diffusion v2 Depth Model", "Realism Engine (v2.1 768)"]
Download_Group = "None" #@param ["None", "Base Models", "OrangeMixs", "Midnight Mixes", "Anything (anime model)"]
#@markdown ##### Choose a model from the dropdown, or choose a group of models to download
#@markdown ### Method 2: Download from URL
Models_from_URL = "" #@param {type:"string"}
#@markdown ##### Fill this with download links (ex: [huggingface](https://huggingface.co/), [civitai](https://civitai.com/)). Download multiple models by separating each link with commas
hf_token = ""  # @param {type:"string"}
#@markdown ##### Fill this with your [huggingface token](https://huggingface.co/settings/tokens) if you need to download your private models
Download_VAE = "All" #@param ["None", "All", "Stable Diffusion", "Anime", "Waifu Diffusion"]
#@markdown ##### Choose a VAE here, or select `All` to download all of them


model_urls = {
    "None": "",
    "Stable Diffusion v1.5": ["https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt"],
    "Stable Diffusion v1.5 (inpainting)": ["https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt"],
    "Realistic Vision v2.0": ["https://civitai.com/api/download/models/29460"],
    "ChilloutMix": ["https://civitai.com/api/download/models/11732"],
    "Deliberate": ["https://civitai.com/api/download/models/15236"],
    "Dreamshaper V3": ["https://civitai.com/api/download/models/6202"],
    "Dreamshaper V4": ["https://civitai.com/api/download/models/25067"],
    "Waifu Diffusion 1.3": ["https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt"],
    "Anything V3": ["https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors"],
    "Anything V4.5": ["https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned-fp16.ckpt"],
    "Anything V5": ["https://civitai.com/api/download/models/30163"],
    "Counterfeit V2.5": ["https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_fp16.safetensors"],
    "AbyssOrangeMix2 SFW (OrangeMixs)": ["https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/Pruned/AbyssOrangeMix2_sfw_pruned_fp16_with_VAE.safetensors"],
    "AbyssOrangeMix3 (OrangeMixs)": ["https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors"],
    "AbyssOrangeMix3A1 (OrangeMixs)": ["https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors"],
    "AbyssOrangeMix3A1B (OrangeMixs)": ["https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1B_orangemixs.safetensors"],
    "AbyssOrangeMix3A2 (OrangeMixs)": ["https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A2_orangemixs.safetensors"],
    "AbyssOrangeMix3A3 (OrangeMixs)": ["https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors"],
    "BloodOrangeMix (OrangeMixs)": ["https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/BloodOrangeMix/BloodOrangeMix.safetensors"],
    "Midnight Maple (Midnight Mixes)": ["https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Maple.safetensors"],
    "Midnight Melt (Midnight Mixes)": ["https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Melt.safetensors"],
    "Midnight Mixer Melt (Midnight Mixes)": ["https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Mixer%20Melt.safetensors"],
    "Stable Diffusion v2.1": ["https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt", "https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml|v2-1_512-ema-pruned.yaml"],
    "Stable Diffusion v2.1 768": ["https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt", "https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml|v2-1_768-ema-pruned.yaml"],
    "Stable Diffusion v2 Depth Model": ["https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt", "https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml|512-depth-ema.yaml"],
    "Realism Engine (v2.1 768)": ["https://civitai.com/api/download/models/20414", "https://civitai.com/api/download/models/20414?type=Config"]
}

model_groups = {
    "None": [],
    "Base Models": ["Stable Diffusion v1.5", "Stable Diffusion v1.5 (inpainting)", "Stable Diffusion v2.1", "Stable Diffusion v2.1 768", "Stable Diffusion v2 Depth Model"],
    "AbyssOrangeMixs3": ["AbyssOrangeMix3 (OrangeMixs)", "AbyssOrangeMix3A1 (OrangeMixs)", "AbyssOrangeMix3A1B (OrangeMixs)", "AbyssOrangeMix3A2 (OrangeMixs)", "AbyssOrangeMix3A3 (OrangeMixs)"],
    "Midnight Mixes": ["Midnight Maple (Midnight Mixes)", "Midnight Melt (Midnight Mixes)", "Midnight Mixer Melt (Midnight Mixes)"],
    "Anything (anime model)": ["Anything V3", "Anything V4.5", "Anything V5"],
    "Dreamshaper": ["Dreamshaper V3", "Dreamshaper V4"]
}

vae_urls = {
    "None": "",
    "All": "",
    "Stable Diffusion": "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt|vae-ft-mse-840000-ema-pruned.ckpt",
    "Anime": "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt|anime.vae.pt",
    "Waifu Diffusion": "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt|waifu-diffusion.ckpt"
}

def downloadModel(url, hf_token=None):
  %cd /content/stable-diffusion-webui/models/Stable-diffusion
  model_url, _, model_name = url.partition("|")
  if 'huggingface.co' in url:
    if not model_name:
      model_name = model_url.split('/')[-1].replace('%20', '_')
    if hf_token:
      user_header = f'"Authorization: Bearer {hf_token}"'
      !aria2c --console-log-level=error --header={user_header} -c -x 16 -s 16 -k 1M {model_url} -o {model_name}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -o {model_name}
  else:
    # civitai
    if model_name:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -o {model_name}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url}

def downloadVAE(url):
  vae_url, _, vae_name = url.partition("|")
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {vae_url} -d /content/stable-diffusion-webui/models/VAE -o {vae_name}

# Might need to clean these up later
if Download_Model in model_urls:
  if Download_Model == "None":
    if not Models_from_URL:
      print("No model specified")
  else:
    for url in model_urls[Download_Model]:
      downloadModel(url)
else:
  print(f'Error: {Download_Model} is not a valid model name')

if Download_Group in model_groups:
  if Download_Group == "None":
    pass
  else:
    for model in model_groups[Download_Group]:
      for url in model_urls[model]:
        downloadModel(url)
else:
  print(f'Error: {Download_Group} is not a valid group')

if Models_from_URL:
  for model in Models_from_URL.split(','):
    if hf_token:
      downloadModel(model, hf_token)
    else:
      downloadModel(model)

if Download_VAE in vae_urls:
  if Download_VAE == "None":
    print("No VAE specified")
  elif Download_VAE == "All":
    vae_list = list(vae_urls.keys())
    vae_list = vae_list[2:]
    for vae in vae_list:
      downloadVAE(vae_urls[vae])
  else:
    downloadVAE(vae_urls[Download_VAE])
else:
  print(f'Error: {Download_VAE} is not a valid VAE')

In [ ]:
#@title ## (Optional) Install Extensions
ControlNet = True #@param {type:"boolean"}
Additional_Networks = True #@param {type:"boolean"}
System_Info = True #@param {type:"boolean"}
UltimateSDUpscale = False #@param {type:"boolean"}
LoCon = False #@param {type:"boolean"}
Aesthetic_Gradients = False #@param {type:"boolean"}
CivitaiHelper = False #@param {type:"boolean"}
CanvasZoom = False #@param {type:"boolean"}
Extensions_from_URL = "" #@param {type:"string"}
#@markdown #### Fill this to install extensions not listed above. Can type the full link or use this format: `user/repo`
#@markdown ##### Full link example: `https://github.com/kohya-ss/sd-webui-additional-networks.git`
#@markdown ##### Short format example: `kohya-ss/sd-webui-additional-networks`

ext_list = [ControlNet, Additional_Networks, UltimateSDUpscale, LoCon, Aesthetic_Gradients, CivitaiHelper, CanvasZoom, System_Info]
ext_install = []
ext_repoindex = {
  0: ["Mikubill/sd-webui-controlnet", "ControlNet"],
  1: ["kohya-ss/sd-webui-additional-networks", "Additional Networks"],
  2: ["Coyote-A/ultimate-upscale-for-automatic1111", "Ultimate SD Upscale"],
  3: ["KohakuBlueleaf/a1111-sd-webui-locon", "LoCon"],
  4: ["AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients", "Aesthetic Gradients"],
  5: ["butaixianran/Stable-Diffusion-Webui-Civitai-Helper", "Civitai Helper"],
  6: ["richrobber2/canvas-zoom", "Canvas Zoom"],
  7: ["vladmandic/sd-extension-system-info", "System Info"]
}

for index, ext in enumerate(ext_list):
  if ext:
    ext_install.append(index)

def installExtensions(ext, ext_displayname=None):
  ext = ext.replace(";", "|")
  ext_commit = None
  ext_link, _, ext_commit = ext.strip().partition("|")
  if not ext_link.startswith("https://") and not ext_link.startswith("http://"):
    if not "github.com" in ext_link:
      ext_link = "github.com/" + ext_link
    ext_link = "https://" + ext_link
  ext_link = ext_link.replace("http://", "https://")
  if not ext_link.endswith('.git'):
    ext_link += '.git'
  ext_name = ext_link.split("/")[-1].split(".")[0]

  print(f"Installing {ext_displayname}...") if ext_displayname else print(f"Installing {ext_name}...")
  !git clone {ext_link}
  %cd {ext_name}
  if ext_commit:
    !git checkout -f {ext_commit}
  if os.path.exists("requirements.txt"):
    print("Installing packages for", ext_displayname) if ext_displayname else print("Installing packages for", ext_name)
    !pip install -r requirements.txt
  print(ext_name, "Install completed.")

if ext_install:
  for index in ext_install:
    %cd /content/stable-diffusion-webui/extensions
    installExtensions(ext_repoindex[index][0], ext_repoindex[index][1])
    if index == 0: # ControlNet
      %cd /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
    if index == 4: # Aesthetic Gradients
      if os.path.exists("/content/drive"): # Check if Google Drive is mounted
        !mkdir -p {output_path}/aesthetic_embeddings
        %cd /content/stable-diffusion-webui/extensions/stable-diffusion-webui-aesthetic-gradients
        !rm -rf aesthetic_embeddings
        !ln -s {output_path}/aesthetic_embeddings

if Extensions_from_URL:
  for ext in Extensions_from_URL.split(','):
    %cd /content/stable-diffusion-webui/extensions
    installExtensions(ext)

# 3. Run the WebUI

In [ ]:
#@title ## Run this cell and wait until a Gradio/NGROK link appear. Click it to open the WebUI
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
#@markdown ##### (Optional) Adds a login page.
NGROK = "" #@param {type:"string"}
#@markdown ##### (Optional) Fill this with your NGROK authtoken. Get them from [here.](https://dashboard.ngrok.com/get-started/your-authtoken)
optimization_method = "opt-sdp-attention" #@param ["none", "xformers", "opt-sdp-attention", "opt-sdp-no-mem-attention"]
#@markdown ##### Select which optimization to use. Note: `opt-sdp-attention` and `opt-sdp-no-mem-attention` only works with PyTorch 2.0.0. Select `xformers` if you're using PyTorch 1.13.1.
args = "--enable-insecure-extension-access --no-half-vae " #@param {type:"string"}
#@markdown ##### Fill this with launch arguments as needed. Go [here](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings) for a list of arguments.

if optimization_method == "none":
  pass
else:
  args += f' --{optimization_method} '
if NGROK:
  args += f' --ngrok {NGROK} '
else:
  args += ' --share '
if username and password:
  args += f' --gradio-auth {username}:{password} '

%cd /content/stable-diffusion-webui
!python launch.py {args}